In [1]:
!pip install pipenv

In [2]:
!pipenv --version

pipenv, version 2024.3.0


In [3]:
!pipenv lock --clear
!pipenv install --skip-lock scikit-learn==1.5.2

Locking [packages] dependencies...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠸ Locking packages...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (adb15e5c5a21f13e221698df1bb36dc8d3bee16bb778fdfcad74b126616757b9)!
The flag --skip-lock has been reintroduced (but is not recommended).  Without 
the lock resolver it is difficult to manage multiple package indexes, and hash 
checking is not provided.  However it can help manage installs with current 
deficiencies in locking across platforms.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.2...
✔ Installation Succeeded
Installing dependencies from Pipfile...


In [4]:
!pipenv run python -c "import sklearn; print(sklearn.__version__)"

1.5.2


In [5]:
!brew install wget

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 1.24.5, run:
  brew reinstall wget


In [6]:
PREFIX="https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
!wget {PREFIX}/model1.bin
!wget {PREFIX}/dv.bin

--2024-10-30 22:12:20--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 850 [application/octet-stream]
Saving to: ‘model1.bin.3’

model1.bin.3        100%[===================>]     850  --.-KB/s    in 0s      

2024-10-30 22:12:20 (17,2 MB/s) - ‘model1.bin.3’ saved [850/850]

--2024-10-30 22:12:20--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443...

In [7]:
!cat Pipfile.lock | grep -A 2 "scikit-learn"

        "scikit-learn": {
            "hashes": [
                "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",


In [8]:
import pickle
!pip install scikit-learn==1.5.2 #not sure why 1.5.2 from above not used, will check later
with open("dv.bin", "rb") as dv_file:
    dv = pickle.load(dv_file)

with open("model1.bin", "rb") as model_file:
    model = pickle.load(model_file)

In [9]:
# Client data
client = {"job": "management", "duration": 400, "poutcome": "success"}

# Transform the client data
X_client = dv.transform([client])

# Predict the probability of subscription
subscription_prob = model.predict_proba(X_client)[0, 1]  # Probability for the positive class
print("Probability of subscription:", subscription_prob)

Probability of subscription: 0.7590966516879658


In [24]:
with open("app.py", "w") as f:
    f.write("""
from flask import Flask, request, jsonify
import pickle

# Load the model and vectorizer
with open("dv.bin", "rb") as f_dv:
    dv = pickle.load(f_dv)

with open("model1.bin", "rb") as f_model:
    model = pickle.load(f_model)

# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    client = request.get_json()
    X_client = dv.transform([client])
    subscription_prob = model.predict_proba(X_client)[0, 1]
    return jsonify({"subscription_probability": subscription_prob})

if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=9696)
""")


In [22]:
pwd

'/Users/pavloskorodziievskyi'

In [26]:
#gunicorn --bind 0.0.0.0:9696 app:app  - launched in separate (non jupyter terminal)

SyntaxError: invalid syntax (708474007.py, line 1)

In [28]:
import requests

url = "http://127.0.0.1:9696/predict"
client = {"job": "student", "duration": 280, "poutcome": "failure"}

response = requests.post(url, json=client).json()
print("Subscription Probability:", response["subscription_probability"])

Subscription Probability: 0.33480703475511053


In [30]:
!docker pull svizor/zoomcamp-model:3.11.5-slim
!docker images svizor/zoomcamp-model:3.11.5-slim


3.11.5-slim: Pulling from svizor/zoomcamp-model

e7c4b030: Pulling fs layer 
36e84c8e: Pulling fs layer 
b60fbb0c: Pulling fs layer 
e97f4fbd: Pulling fs layer 
d4e17ae3: Pulling fs layer 
f8d7a667: Pulling fs layer 
Digest: sha256:15d61790363f892dfdef55f47b78feed751cb59704d47ea911df0ef3e9300c06[5A
Status: Downloaded newer image for svizor/zoomcamp-model:3.11.5-slim
docker.io/svizor/zoomcamp-model:3.11.5-slim

What's next:
    View a summary of image vulnerabilities and recommendations → docker scout quickview svizor/zoomcamp-model:3.11.5-slim
REPOSITORY              TAG           IMAGE ID       CREATED       SIZE
svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   11 days ago   130MB


In [77]:
with open("Dockerfile", "w") as f:
    f.write("""
    FROM svizor/zoomcamp-model:3.11.5-slim

    # Install required packages
    RUN pip install gunicorn flask

    # Set working directory
    WORKDIR /app

    # Copy Pipenv files and install dependencies
    COPY Pipfile Pipfile.lock /app/
    RUN pip install pipenv && pipenv install --system --deploy

    # Copy the application and model files
    COPY app.py model1.bin dv.bin /app/

    # Expose the port for the app
    EXPOSE 9696

    # Start the app with Gunicorn
    CMD ["gunicorn", "--bind", "0.0.0.0:9696", "app:app"]
    """)


In [79]:
!docker build -t ps_image .

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 578B                                       0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.11.5-s  0.2s
[+] Building 0.4s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 578B                                       0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.11.5-s  0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 578B                                      

In [81]:
!docker images ps_image

REPOSITORY   TAG       IMAGE ID       CREATED         SIZE
ps_image     latest    5cc76724437e   5 minutes ago   546MB


In [ ]:
!docker run -p 9696:9696 ps_image

[2024-10-30 21:46:10 +0000] [1] [INFO] Starting gunicorn 23.0.0
[2024-10-30 21:46:10 +0000] [1] [INFO] Listening at: http://0.0.0.0:9696 (1)
[2024-10-30 21:46:10 +0000] [1] [INFO] Using worker: sync
[2024-10-30 21:46:10 +0000] [7] [INFO] Booting worker with pid: 7


In [ ]:
!docker build --platform linux/arm64 -t ps_image .

In [ ]:
!docker run --platform linux/amd64 -p 9696:9696 ps_image

In [ ]:
!docker ps -a

In [ ]:
import requests

# Define the URL of the running Docker container
url = "http://127.0.0.1:9696/predict"

client = {
    "job": "management",
    "duration": 400,
    "poutcome": "success"
}

response = requests.post(url, json=client)

if response.status_code == 200:
    result = response.json()
    print("Subscription Probability:", result["subscription_probability"])
else:
    print("Error:", response.status_code, response.text)


In [ ]:
#tested in browser console with:
fetch("http://127.0.0.1:9696/predict", {
    method: "POST",
    headers: {
        "Content-Type": "application/json"
    },
    body: JSON.stringify({
        "job": "management",
        "duration": 400,
        "poutcome": "success"
    })
})
.then(response => response.json())
.then(data => console.log("Subscription Probability:", data.subscription_probability))
.catch(error => console.error("Error:", error));